# DES DR2 Paper:  Saturation Limits

Created:  2021.04.20

Calculate the bright (saturation) limit for a point source (e.g., star) in the DES DR2, assuming the centroid of the point source is centered in a pixel and taking typical values for the pixel full well and for the photometric zeropoints and seeing for an exposure observed in typical (median) and in poor conditions.

## 1. Initial Setup...

In [1]:
import numpy as np
import pandas as pd
import glob
import math
import os

import matplotlib.pyplot as plt


%matplotlib inline

## 2. Typical Seeing (for wide survey)

From Robert Gruendl (2021.04.22):

<pre>
select e.band,to_char(median(q.psf_fwhm),'9.99'),
       to_char(max(q.psf_fwhm),'9.99') 
from y6a1_proctag t, y6a1_qa_summary q, y6a1_exposure e 
where t.tag='Y6A1_COADD_INPUT' and 
      t.pfw_attempt_id=q.pfw_attempt_id and 
      q.expnum=e.expnum
group by e.band 
order by decode(e.band,'u',0,'g',1,'r',2,'i',3,'z',4,'Y',5) asc;

BAND  TO_CH TO_CH
----- ----- -----
g      1.09  1.72
r      0.95  1.63
i      0.88  1.56
z      0.84  1.50
Y      0.89  1.72
</pre>

## 3. Exposure List (with relatively high Teff) 

From Robert Gruendl (2021.04.22):

<pre>
with x as (
    select e.band,max(q.t_eff) as maxteff
    from y6a1_proctag t, y6a1_qa_summary q, y6a1_exposure e, tmp_typical_seeing s
    where t.tag='Y6A1_COADD_INPUT'
        and t.pfw_attempt_id=q.pfw_attempt_id
        and q.expnum=e.expnum and e.band=s.band
        and abs(q.psf_fwhm-s.medval)<0.001
        and q.t_eff between 0.9 and 1.0
    group by e.band),
y as (select e.band,max(q.t_eff) as maxteff
    from y6a1_proctag t, y6a1_qa_summary q, y6a1_exposure e, tmp_typical_seeing s
    where t.tag='Y6A1_COADD_INPUT'
        and t.pfw_attempt_id=q.pfw_attempt_id
        and q.expnum=e.expnum
        and e.band=s.band
        and abs(q.psf_fwhm-s.maxval)<0.001
        and q.t_eff>0.3
    group by e.band)
select x.band,e.expnum,f.expnum
from x, y, y6a1_exposure e, y6a1_exposure f, y6a1_qa_summary qe, y6a1_qa_summary qf
where x.band=y.band
    and qe.t_eff=x.maxteff
    and qe.expnum=e.expnum
    and e.band=x.band
    and qf.t_eff=y.maxteff
    and qf.expnum=f.expnum
    and f.band=y.band
order by decode(x.band,'u',0,'g',1,'r',2,'i',3,'z',4,'Y',5) asc;

BAND	  EXPNUM     EXPNUM
----- ---------- ----------
g	  791360     799382
r	  791366     798969
i	  578292     254305
z	  499048     570235
Y	  668721     573010
</pre>

## 4. Pixel Saturation Limit and Photometric Zeropoint

From Robert Gruendl (2021.04.22):

<pre>
            MEDIAN SEEING                  WORST SEEING
BAND    EXPNUM  SATURATE   ZPT        EXPNUM  SATURATE  ZPT
-----   ------ --------- --------     ------ --------- --------
 g      791360  179909.5  31.5924     799382  179909.5  31.5380
 r      791366  179515.4  31.7503     798969  179515.4  31.6884
 i      578292  180345.0  31.6751     254305  182071.8  31.6630
 z      499048  180552.1  31.3209     570235  181146.4  31.4067
 Y      668721  179150.8  29.5498     573010  180202.5  29.4741
</pre>

## 5. Input Info

In [2]:
bandList = [ 'g', 'r', 'i', 'z', 'Y']

# DECam pixel scale...
pixScale = 0.264          # in arcsec/pixel

# Use the median seeing values or the "worst" seeing values...
use_med_seeing = True

if use_med_seeing == True:

    # Values for median seeing exposures...

    # Median seeing for single-epoch exposures used in DES DR2 (in arcsec)
    #  (from R. Gruendl's query above)...
    seeing_dict = { 
                    'g':1.09,
                    'r':0.95,
                    'i':0.88,
                    'z':0.84,
                    'Y':0.89
    }   

    # Exposures with near-median seeing 
    #  (from R. Gruendl's query above)...
    expnum_dict = { 
                    'g':791360,
                    'r':791366,
                    'i':578292,
                    'z':499048,
                    'Y':668721
    }

    # Exposure times from exposures in expnum_dict
    #  (queried separately from desoper)...
    exptime_dict = { 
                    'g':90.,
                    'r':90.,
                    'i':90.,
                    'z':90.,
                    'Y':45.
    }
        
    # Pixel full-well saturation, medianed over all CCDs in exposure (in electrons) 
    #  (from R. Gruendl's query above)...
    satcounts_dict = {
                    'g':179909.5,
                    'r':179515.4,
                    'i':180345.0,
                    'z':180552.1,
                    'Y':179150.8,
    }

    # FGCM photometric zeropoints, medianed over all CCDs in exposure (in mags)
    #  (from R. Gruendl's query above)...
    zpt_dict = {
                    'g':31.5924,
                    'r':31.7503,
                    'i':31.6751,
                    'z':31.3209,
                    'Y':29.5498 
    }
    
    # Single-epoch median sky brightness (in mag/arcsec**2)
    #  (from Table 1 of DES DR2 table)...
    skybright_dict = {
                    'g':22.05,
                    'r':21.18,
                    'i':19.92,
                    'z':18.74,
                    'Y':17.97 
    }
    
else:
    
    # Values for "worst" seeing exposures...

    # "Worst" seeing for single-epoch exposures used in DES DR2 (in arcsec)
    #  (from R. Gruendl's query above)...
    seeing_dict = { 
                    'g':1.72,
                    'r':1.63,
                    'i':1.56,
                    'z':1.50,
                    'Y':1.72
    }   

    # Exposures with near-"worst" seeing 
    #  (from R. Gruendl's query above)...
    expnum_dict = { 
                    'g':799382,
                    'r':798969,
                    'i':254305,
                    'z':570235,
                    'Y':573010
    }

    # Exposure times from exposures in expnum_dict
    #  (queried separately from desoper)...
    exptime_dict = { 
                    'g':90.,
                    'r':90.,
                    'i':90.,
                    'z':90.,
                    'Y':45.
    }

    # Pixel full-well saturation, medianed over all CCDs in exposure (in electrons) 
    #  (from R. Gruendl's query above)...
    satcounts_dict = {
                    'g':179909.5,
                    'r':179515.4,
                    'i':182071.8,
                    'z':181146.4,
                    'Y':180202.5
    }

    # FGCM photometric zeropoints, medianed over all CCDs in exposure (in mags)
    #  (from R. Gruendl's query above)...
    zpt_dict = {
                    'g':31.5380,
                    'r':31.6884,
                    'i':31.6630,
                    'z':31.4067,
                    'Y':29.4741
    }
    
    # Single-epoch median sky brightness (in mag/arcsec**2)
    #  (from Table 1 of DES DR2 table)...
    skybright_dict = {
                    'g':22.05,
                    'r':21.18,
                    'i':19.92,
                    'z':18.74,
                    'Y':17.97 

    }


## 6. Some Function Definitions

In [3]:
# Calculate the fraction of light within a circular aperture
#  assuming the light profile is a 2d Gaussian and the centroid
#  of the Gaussian profile is at the center of the aperture...
def gaussFracCircAper(seeing, apRadius):
    import math
    sigma = seeing/2.354
    # Integral of the the Gaussian profile; see, e.g., K. Mighell 1999a):
    fracCircAper = 1. - math.exp(-0.5*apRadius*apRadius/(sigma*sigma))
    return fracCircAper

# Roughly estimate the fraction of light within a single square
#  pixel, assuming the light profile is a 2d Gaussian and the 
#  centroid of the Gaussian profile is at the center of the pixel.
def gaussFracSinglePixel(seeing, pixScale):
    import math
    apRadius = 0.5*pixScale
    fracCircAper = gaussFracCircAper(seeing, apRadius)
    # The correction factor 4./pi is an approximation, 
    #  to help take into account fact that aperture is 
    #  square, not circular.  This is only a very rough
    #  approximation, but should be good enough for our  
    #  purposes here...
    fracSinglePixel = (4./math.pi)*fracCircAper
    return fracSinglePixel

## 7. Estimates

In [4]:
# Loop over each band...

for band in bandList:
    
    # Extract values from Python dictionaries instantiated above...
    seeing = seeing_dict[band]
    expnum = expnum_dict[band]
    exptime = exptime_dict[band]
    satcounts = satcounts_dict[band]
    zpt = zpt_dict[band]
    skybright = skybright_dict[band]
    
    # (Roughly) calculate the fraction of flux from a point source
    #  in a pixel, assuming that the point source flux has a 2d Gaussian
    #  profile, and that the point source is centered on the pixel...
    fracAper1pix = gaussFracSinglePixel(seeing, pixScale)

    # Counts for a 20th mag object in electrons...
    counts20 = math.pow(10., -0.4*(20.00-zpt))
    # Count rate for a 20th mag object in electrons/sec...
    countrate20 = counts20/exptime

    # Sky counts in electrons/arcsec**2
    skycounts = math.pow(10., -0.4*(skybright-zpt))
    # Sky counts in electrons/pixel
    skycounts = skycounts*pixScale*pixScale
    # Sky count rate in electrons/pixel/second
    skycountrate = skycounts/exptime
    
    #skycountrate = 0.00
    numer = satcounts - exptime*skycountrate
    denom = exptime*fracAper1pix*countrate20 
    ratio = numer/denom
    mag_sat = -2.5*math.log10(ratio) + 20.0

    print band, seeing, expnum, exptime, satcounts, zpt, \
             countrate20, skybright, skycountrate, fracAper1pix, \
             mag_sat
        


g 1.09 791360 90.0 179909.5 31.5924 481.634391554 22.05 5.080720969 0.0506984242197 15.2200139728
r 0.95 791366 90.0 179515.4 31.7503 557.028595457 21.18 13.0943921891 0.066317778294 15.6762718599
i 0.88 578292 90.0 180345.0 31.6751 519.753581411 19.92 38.9946785218 0.0769500600753 15.7716989006
z 0.84 499048 90.0 180552.1 31.3209 375.07445817 18.74 83.4305956643 0.0841996315063 15.5387882144
Y 0.89 668721 45.0 179150.8 29.5498 146.79372333 17.97 66.3613694972 0.0752824385036 13.6267379698


### 7.a.  Median Conditions
<pre>
band seeing expnum exptime satcounts zpt countrate20 skybright skycountrate fracAper1pix mag_sat
g 1.09 791360 90.0 179909.5 31.5924 481.634391554 22.05  5.080720969  0.0506984242197 15.2200139728
r 0.95 791366 90.0 179515.4 31.7503 557.028595457 21.18 13.0943921891 0.066317778294  15.6762718599
i 0.88 578292 90.0 180345.0 31.6751 519.753581411 19.92 38.9946785218 0.0769500600753 15.7716989006
z 0.84 499048 90.0 180552.1 31.3209 375.07445817  18.74 83.4305956643 0.0841996315063 15.5387882144
Y 0.89 668721 45.0 179150.8 29.5498 146.79372333  17.97 66.3613694972 0.0752824385036 13.6267379698
</pre>

### 7.b.  "Worst" Conditions 
<pre>
band seeing expnum exptime satcounts zpt countrate20 skybright skycountrate fracAper1pix mag_sat
g 1.72 799382 90.0 179909.5 31.538  458.097045248 22.05  4.83242746872 0.0206084515558 14.1881060308
r 1.63 798969 90.0 179515.4 31.6884 526.159590152 21.18 12.3687366927  0.0229258843135 14.4607146439
i 1.56 254305 90.0 182071.8 31.663  513.993338629 19.92 38.5625144665  0.0250087060133 14.5285172818
z 1.50 570235 90.0 181146.4 31.4067 405.917234038 18.74 90.2911832266  0.0270276078385 14.3909770386
Y 1.72 573010 45.0 180202.5 29.4741 136.90757759  17.97 61.8921173013  0.0206084515558 12.1367231551
</pre>

## 8.  Some Comparison Values from DES SV tests

Some earlier values, from DES Science Verification tests, as a comparison....
 
<pre>
# Some constants    
counts1pixSat = 130000.      # Nominal saturation at 130000 electrons...
pixScale     = 0.27          # in arcsec/pixel
fracAper     = 0.93739       # appropriate for an apRadius=1.0*FWHM
fracAper1pix = 0.07695       # appropriate for a single pixel
apRadius     = 0.9           # in arcsec
npix = math.pi*(apRadius/pixScale)*(apRadius/pixScale)

# From Huan Lin's 11 Sep 2012 e-mail:
nstar_u =  92.  # in e-/sec for a mag=20 star
nstar_g = 590.
nstar_r = 588.
nstar_i = 566.
nstar_z = 446.
nstar_Y = 159.

# From DES-doc##3091 (except for u, which is a guess).
nsky_u =  2.    # in e-/sec/pixel
nsky_g =  5.8
nsky_r = 13.6
nsky_i = 25.4
nsky_z = 73.6
nsky_Y = 73.2

</pre>